In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()

In [7]:
!pip install pyspark==3.0.1

     |████████████████████████████████| 204.2 MB 94 kB/s s eta 0:00:01
  Using cached py4j-0.10.9-py2.py3-none-any.whl (198 kB)
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=14461dde0fce484754293478ff4bab4200fda297a6fa43e87388e512a33c50fb
  Stored in directory: /Users/violetamilanovic/Library/Caches/pip/wheels/ea/21/84/970b03913d0d6a96ef51c34c878add0de9e4ecbb7c764ea21f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.7
    Uninstalling py4j-0.10.7:
      Successfully uninstalled py4j-0.10.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 2.4.4
    Uninstalling pyspark-2.4.4:
      Successfully uninstalled pyspark-2.4.4


In [5]:
!python --version

Python 3.8.8


In [10]:
#ransactions  = spark.read.option("escape","\"").option("multiline",True).csv("./trans.csv", header=True, sep=",")

In [14]:
#products  = spark.read.option("escape","\"").option("multiline",True).csv("../input/dropped_dup.csv", header=True, sep=",")
customers  = spark.read.option("escape","\"").option("multiline",True).csv("../input/customers.csv", header=True, sep=",")
transactions  = spark.read.option("escape","\"").option("multiline",True).csv("../input/transactions.csv", header=True, sep=",")#

In [4]:
products_wc = spark.read.option("escape","\"").option("multiline",True).csv("./products2.csv", header=True, sep="|")

In [16]:
#products.createOrReplaceTempView("products")
#customers.createOrReplaceTempView("customers")
transactions.createOrReplaceTempView("transactions")

+--------------+----------+------------------+-------------------+-----------+
|transaction_id|product_id|transaction_amount|   transaction_date|customer_id|
+--------------+----------+------------------+-------------------+-----------+
|        172161|9742356831|             49.01|2020-02-08 08:09:39|     754658|
|        155162|B00005344V|            100.43|2019-12-29 09:56:46|     945224|
|        248916|B000052X2S|            491.91|2020-05-02 07:10:24|     061595|
|        477300|5463213682|            215.05|2020-03-19 08:00:03|     061595|
|        926780|9742356831|            690.79|2020-02-02 05:12:57|     033396|
+--------------+----------+------------------+-------------------+-----------+
only showing top 5 rows



In [240]:
transactions.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- transaction_amount: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- customer_id: string (nullable = true)



In [19]:
spark.sql("""
    select 
    avg(cast(transaction_amount as double)) as avg_ta,
    min(cast(transaction_amount as double)) as min_ta,
    max(cast(transaction_amount as double)) as max_ta,
    sum(cast(transaction_amount as double)) as sum_ta
    from transactions""").show()

+----------------+------+------+------------------+
|          avg_ta|min_ta|max_ta|            sum_ta|
+----------------+------+------+------------------+
|251.723409599998|   0.0|992.26|1.25861704799999E7|
+----------------+------+------+------------------+



In [233]:
import pyspark.sql.functions as F
transactions.agg(F.max(F.abs(transactions.transaction_amount))).first()[0]

929.07

In [252]:
spark.sql("""select *
        from transactions t left join products p on t.product_id=p.asin
        left join customers c on t.customer_id=c.customer_id""").show()

+--------------+----------+------------------+-------------------+-----------+----------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+----------+---------+------+-----------+-------------------+--------------------+----------+----------+---------------+--------+--------------+------+-----+----------------+
|transaction_id|product_id|transaction_amount|   transaction_date|customer_id|      asin|            category|               title|               brand|                rank|   main_cat|customer_id|first_name|last_name|gender|        ssn|        credit_card|credit_card_provider|birth_date|start_date|          title|  office|  organization|salary|bonus|accured_holidays|
+--------------+----------+------------------+-------------------+-----------+----------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+----------+---------+------+-----------+-------------------+-

In [261]:
spark.sql("""
        select 
        count(customer_id) as customers_no,
        first(title) as title, 
        product_id 
        from transactions t left join products p on t.product_id = p.asin
        group by product_id""").show()

+------------+--------------------+----------+
|customers_no|               title|product_id|
+------------+--------------------+----------+
|          40|WHOPPERS Candy (C...|B00005BPQ9|
|          38|Ricola Cough Supp...|B000052X2S|
|           3|Starbucks Tribute...|7621000880|
|         274|Traditional Medic...|B00005344V|
|          88|Metalware Open Co...|B00004W4VD|
|         234|Mae Ploy Green Cu...|9742356831|
|          82|Organo Gold Cafe ...|5463213682|
|          29|Lipton Yellow Lab...|4639725183|
|           6|Japanese Kit Kat ...|6163058011|
|          68|Lipton Yellow Lab...|4639725043|
|           5|EAS Myoplex Origi...|B000053408|
|          20|Vacu Vin Coffee S...|B00005AS53|
|         100|Ateco Food Colori...|B00004S1C5|
|           5|Letter S - Swarov...|1888861614|
|           8|Biotene Dry Mouth...|B000052Y74|
+------------+--------------------+----------+



In [6]:
spark.sql("""
        select 
        first(c.first_name),
        first(c.last_name),
        avg(transaction_amount) as avg_ta,
        avg(salary+bonus) as avg_income,
        avg(salary+bonus) - avg(transaction_amount) as spending
        from transactions as t left join customers as c on t.customer_id = c.customer_id
        group by c.customer_id""").show(5)

+------------------------+-----------------------+------------------+----------+------------------+
|first(first_name, false)|first(last_name, false)|            avg_ta|avg_income|          spending|
+------------------------+-----------------------+------------------+----------+------------------+
|                   Tonya|                Holland|             346.7|  175500.0|          175153.3|
|                 Natalie|                 Pruitt|314.53142857142865|  254000.0|253685.46857142856|
|                  Hannah|                  Wells|            125.24|   87000.0|          86874.76|
|                   Stacy|                   Ford|241.45200000000006|  138000.0|        137758.548|
|                   Kevin|                 Mclean|            175.47|  163000.0|         162824.53|
+------------------------+-----------------------+------------------+----------+------------------+
only showing top 5 rows



In [254]:
spark.sql("""
        select 
        avg(transaction_amount) as avg_ta,
        avg(salary+bonus) as avg_income,
        avg(salary+bonus) - avg(transaction_amount) as spending
        from transactions t left join customers c  on t.customer_id = c.customer_id""").show()

+------------------+----------+------------+
|            avg_ta|avg_income|    spending|
+------------------+----------+------------+
|250.25398999999973|  129520.0|129269.74601|
+------------------+----------+------------+



In [258]:
spark.sql("""
        select 
        avg(cast(transaction_amount as double)) as avg_ta,
        avg(cast(salary as double) + cast(bonus as double)) as avg_income,
        avg(cast(salary as double) + cast(bonus as double)) - avg(cast(transaction_amount as double)) as spending
        from transactions t left join customers c  on t.customer_id = c.customer_id""").show()

+------------------+----------+------------+
|            avg_ta|avg_income|    spending|
+------------------+----------+------------+
|250.25398999999973|  129520.0|129269.74601|
+------------------+----------+------------+



In [7]:
spark.sql("""
        select
        CONCAT(first(first_name), ' ', first(last_name))  as full_name,
        CONCAT_WS(";",collect_set(p.title)) as products,
        CONCAT_WS(";",collect_set(p.brand)) as product_brand,
        CONCAT_WS(";",collect_set(p.main_cat)) as main_cat,
        CONCAT_WS(";",collect_list(transaction_id)) as transactions,
        MIN(transaction_date) as first_transaction_date,
        avg(cast(transaction_amount as double)) as avg_transaction,
        min(cast(transaction_amount as double)) as min_transaction,
        max(cast(transaction_amount as double)) as max_transaction,
        avg(cast(salary as double)) as avg_salary,
        avg(cast(bonus as double)) as avg_bonus
        from transactions t left join customers c on t.customer_id=c.customer_id
        join products p on t.product_id = p.asin
        group by t.customer_id""").show(5)

+--------------+--------------------+--------------------+-------------------+--------------------+----------------------+------------------+---------------+---------------+----------+---------+
|     full_name|            products|       product_brand|           main_cat|        transactions|first_transaction_date|   avg_transaction|min_transaction|max_transaction|avg_salary|avg_bonus|
+--------------+--------------------+--------------------+-------------------+--------------------+----------------------+------------------+---------------+---------------+----------+---------+
| Tonya Holland|Mae Ploy Green Cu...|Mae Ploy;Traditio...|            Grocery|501147;639364;644...|   2019-11-17 02:44:22|             346.7|          85.72|         779.13|  147000.0|  28500.0|
|Natalie Pruitt|Metalware Open Co...|Whoppers;Vacu Vin...|Amazon Home;Grocery|694730;720988;488...|   2019-08-23 15:27:38|314.53142857142865|          30.28|         769.86|  220000.0|  34000.0|
|  Hannah Wells|Vacu Vin 

In [13]:
import pyspark.sql.functions as f
products_wc.withColumn("words", 
                       f.size(f.split(products_wc['reviewText'],
                                      ' '))).orderBy(f.desc('words')).show()

+-------+--------+----------+--------------------+--------------------+-----+
|overall|verified|      asin|          reviewText|             summary|words|
+-------+--------+----------+--------------------+--------------------+-----+
|    5.0|   False|9742356831|I have two things...|Authentic, easy, ...|  458|
|    4.0|   False|B00005344V|I'm in the medica...|A quality item th...|  399|
|    4.0|   False|B00005344V|I'm in the medica...|A quality item th...|  399|
|    5.0|    True|9742356831|I ordered both th...|Mae Ploy curries ...|  323|
|    5.0|   False|9742356831|As an experienced...|          Very nice!|  269|
|    5.0|   False|B00004W4VD|I have to admit w...|Ground beef jerky...|  263|
|    5.0|   False|B00004W4VD|I have to admit w...|Ground beef jerky...|  263|
|    5.0|   False|B00004W4VD|I am a casual use...|hesitant at first...|  248|
|    5.0|   False|B00004W4VD|I am a casual use...|hesitant at first...|  248|
|    4.0|   False|B00005AS53|After using the w...|Not perfect, b

In [171]:
#### MAIN

products.withColumn('data', f.split('reviewText', ' ')) \
           .withColumn('cnt', f.expr('size(filter(data, x -> x in ("love", "like")))'))\
.orderBy(f.desc('cnt')).show()

+-------+--------+----------+--------------------+--------------------+--------------------+---+
|overall|verified|      asin|          reviewText|             summary|                data|cnt|
+-------+--------+----------+--------------------+--------------------+--------------------+---+
|    3.0|   False|B000052Y74|I like chewing gu...|Dry mouth patient...|[I, like, chewing...|  6|
|    3.0|   False|B000052Y74|I like chewing gu...|Dry mouth patient...|[I, like, chewing...|  6|
|    2.0|    True|5463213682|I like Organo Gol...|Cafe Supreme Not ...|[I, like, Organo,...|  4|
|    5.0|    True|B00005344V|Seems like they m...|   Excellent product|[Seems, like, the...|  4|
|    3.0|    True|9742356831|This green curry ...|Hope you like it ...|[This, green, cur...|  4|
|    5.0|    True|B00005344V|Seems like they m...|   Excellent product|[Seems, like, the...|  4|
|    2.0|   False|9742356831|My husband likes ...|Too much red chil...|[My, husband, lik...|  3|
|    5.0|    True|4639725043|T

In [154]:
mm.count("love")

1

In [152]:
mm = "I like Organo Gold products.  I was an Organo Gold distributor.  However, I wasn't interested in all the hype and the going to this meeting, that meeting, this event, and more.  However, I really liked the coffee though I am not a coffee drinker.  I drink the Mocha and Latte.  I purchased the Cafe Supreme just to try something different.  I didn't like the taste.  It tasted like it had burnt wood shavings in it.  Perhaps a coffee drinker would love that taste.  I didn't.|Cafe Supreme Not Good For Me"

In [155]:
mm.count("like")

4

In [185]:
dd=products.withColumn('count', f.size(f.split(f.col("reviewText"), "like|love"))-1).orderBy(f.desc('count'))

In [173]:
dd.show(5, truncate=True)

+-------+--------+----------+--------------------+--------------------+-----+
|overall|verified|      asin|          reviewText|             summary|count|
+-------+--------+----------+--------------------+--------------------+-----+
|    3.0|   False|B000052Y74|I like chewing gu...|Dry mouth patient...|    7|
|    3.0|   False|B000052Y74|I like chewing gu...|Dry mouth patient...|    7|
|    2.0|    True|5463213682|I like Organo Gol...|Cafe Supreme Not ...|    5|
|    5.0|   False|B00005344V|I love, love, lov...|Licorice action i...|    5|
|    5.0|   False|B00005344V|I love, love, lov...|Licorice action i...|    5|
+-------+--------+----------+--------------------+--------------------+-----+
only showing top 5 rows



In [162]:
stri = "I like chewing gum for my dry mouth, but Biotene is not my favorite: it is expensive, hard to find, and, in my opinion, not any better than many traditional chewing gums.  You have to chew two and a time, and that costs about $.20 per use at the listed price, not including shipping. The gum is boring and small and doesn't retain flavor or the texture I like, so I don't chew it very long when I use it. I don't like hard gums like Dentine or bubble gum; I usually use a soft sugar free gum like Extra.  It's cheap and comes in lots of flavors and stimulates saliva flow. I'm not a big chewer, I mostly suck on it, or let it sit in my mouth.  Biotene may be better in terms of mildness, but I love flavored gum, and am not willing to shell out so much to keep buying Biotene gum. (I do like Biotene toothpaste , though.)|Dry mouth patient reviews Biotene gum."

In [169]:
stri.count("love")

1

In [215]:
#regex = "\\d+(\\.\\d+)?%"
regex = '\d+(?:\.\d+)?%'

In [222]:
%%time
reg=products_wc.withColumn('count', f.size(f.split(f.col("reviewText"), regex))-1).orderBy(f.desc('count'))

CPU times: user 1.97 ms, sys: 1.82 ms, total: 3.79 ms
Wall time: 76.6 ms


In [180]:
#reg.show(5, truncate=False)

In [223]:
reg.show()

+-------+--------+----------+--------------------+--------------------+-----+
|overall|verified|      asin|          reviewText|             summary|count|
+-------+--------+----------+--------------------+--------------------+-----+
|    5.0|    True|B00005344V|I love the tradit...|Traditional Medic...|    3|
|    5.0|    True|9742356831|Last year I spent...|Great & authentic...|    1|
|    5.0|   False|B00004W4VD|I am a casual use...|hesitant at first...|    1|
|    5.0|   False|9742356831|This green curry ...|    Can not go wrong|    1|
|    5.0|   False|B00004W4VD|I am a casual use...|hesitant at first...|    1|
|    5.0|    True|B00005344V|I love the tradit...|Traditional Medic...|    1|
|    5.0|    True|B00005344V|I love the tradit...|Traditional Medic...|    1|
|    4.0|   False|9742356831|I thought the fla...|      Good but salty|    1|
|    4.0|    True|9742356831|pretty good curry...|          good curry|    1|
|    5.0|    True|1888861614|Very pleased with...|             L

In [221]:
products.show(15, truncate=False)

+-------+--------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------+
|overall|verified|asin      |reviewText                                                                                                                                                                              |summary                                                          |
+-------+--------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------+
|5.0    |True    |1888861614|Very pleased with my purchase. Looks exactly like the picture and will look great on my cake. It definitely will sparkle.       